In [40]:
from cassandra.cluster import Cluster

cluster = Cluster(['ec2-54-210-40-33.compute-1.amazonaws.com'])

session = cluster.connect()

In [41]:
from sys import stdout

In [42]:
import pandas as pd

In [43]:
# Import the necessary package to process data in JSON format
try:
    import json
except ImportError:
    import simplejson as json

# Import the necessary methods from "twitter" library
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream

# Variables that contains the user credentials to access Twitter API
ACCESS_TOKEN = '243404150-gpJjlAII45RoGQvFmbw3X7zBB7clv0VBcfCuu3Za'
ACCESS_SECRET = 'd3KllH1qmUKvyWYhszcslZh0mwfvMSXhLWS1N9Vv2o0GQ'
CONSUMER_KEY = 'Uj8qQJ1D6Wj1xyib4UVw'
CONSUMER_SECRET = '70iwVA0LR8Lz6DeYSibzSlAgHggnJZgg32xnK6tBdzs'

oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

In [44]:
# Initiate the connection to Twitter Streaming API
twitter_stream = TwitterStream(auth=oauth)

In [46]:
tweet_ddl = '''CREATE KEYSPACE twitter 
        WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : 3};
    USE twitter;
    CREATE TABLE tweets (
        tweet_id bigint PRIMARY KEY,
        created_at timestamp,
        user_name varchar,
        tweet_text text,
        favorite_count int,
        retweet_count int,
        entities text);'''
if False:
    session.execute(tweet_ddl)

In [47]:
iterator = twitter_stream.statuses.sample()

In [48]:
tweet_count = 0
for tweet in iterator:
    if 'id' in tweet:
        tweet_count += 1
        row = (tweet['id'], str(pd.to_datetime(tweet.get('created_at'))),
               tweet['user'].get('name'), tweet.get('text'),
               tweet.get('favorite_count'), tweet.get('retweet_count'),
               json.dumps(tweet.get('entities')))
        session.execute("""INSERT INTO twitter.tweets 
                (tweet_id, created_at, user_name, tweet_text, favorite_count, retweet_count, entities) 
            VALUES (%s,%s,%s,%s,%s,%s,%s)""", row)
    if tweet_count > 99:
        stdout.write('.')
        tweet_count = 0

........................................

In [32]:
tweet_count

25

Traceback (most recent call last):
  File "cassandra/cluster.py", line 2623, in cassandra.cluster.ControlConnection._reconnect_internal (cassandra/cluster.c:47899)
    return self._try_connect(host)
  File "cassandra/cluster.py", line 2645, in cassandra.cluster.ControlConnection._try_connect (cassandra/cluster.c:48416)
    connection = self._cluster.connection_factory(host.address, is_control_connection=True)
  File "cassandra/cluster.py", line 1119, in cassandra.cluster.Cluster.connection_factory (cassandra/cluster.c:15085)
    return self.connection_class.factory(address, self.connect_timeout, *args, **kwargs)
  File "cassandra/connection.py", line 324, in cassandra.connection.Connection.factory (cassandra/connection.c:5504)
    conn = cls(host, *args, **kwargs)
  File "/Users/alessandro/anaconda/envs/dsci6007/lib/python2.7/site-packages/cassandra/io/asyncorereactor.py", line 299, in __init__
    self._connect_socket()
  File "cassandra/connection.py", line 363, in cassandra.connecti

In [49]:
sc = pyspark.SparkContext('ec2-54-164-67-97.compute-1.amazonaws.com')

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Could not parse Master URL: 'ec2-54-164-67-97.compute-1.amazonaws.com'
	at org.apache.spark.SparkContext$.org$apache$spark$SparkContext$$createTaskScheduler(SparkContext.scala:2499)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:492)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:422)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:240)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:236)
	at py4j.commands.C